In [ ]:
        "source": [
                "import json",
                "",
                "def normalize_medical_qa(input_file, output_file=\"normalized_medical_qa.json\"):",
                "    # 1. Carrega o dataset médico",
                "    with open(input_file, \"r\", encoding=\"utf-8\") as file:",
                "        data = json.load(file)",
                "",
                "    # Suporta formatos: dict (id -> record) ou lista de registros",
                "    qas = []",
                "    if isinstance(data, dict):",
                "        for _id, rec in data.items():",
                "            question = rec.get(\'question\') or rec.get(\'QUESTION\') or ''",
                "            answer = rec.get(\'LONG_ANSWER\') or rec.get(\'long_answer\') or rec.get(\'answer\') or rec.get(\'ANSWER\') or ''",
                "            qas.append({\'question\': question, \'answer\': answer})",
                "    elif isinstance(data, list):",
                "        for rec in data:",
                "            question = rec.get(\'question\') or rec.get(\'QUESTION\') or ''",
                "            answer = rec.get(\'LONG_ANSWER\') or rec.get(\'long_answer\') or rec.get(\'answer\') or rec.get(\'ANSWER\') or ''",
                "            qas.append({\'question\': question, \'answer\': answer})",
                "    else:",
                "        raise ValueError(\"Formato de JSON não esperado: esperado dict ou list\")",
                "",
                "    normalized_results = []",
                "",
                "    # 2. Processa cada pergunta e resposta",
                "    for item in qas:",
                "        question = item['question']",
                "        answer = item['answer']",
                "",
                "        if not question and answer:",
                "            question = answer[:80] + '...'",
                "",
                "        # Chamada para a API (ajuste se necessário)",
                "        response = client.chat.completions.create(",
                "            model=\"gpt-3.5-turbo\",",
                "            response_format={\"type\": \"json_object\"},",
                "            messages=[",
                "                {",
                "                    \"role\": \"system\",",
                "                    \"content\": (",
                "                        \"You are a clinical decision-support assistant. \"",
                "                        \"Rewrite the medical answer in a concise, objective, and clinically safe way. \"",
                "                        \"Do NOT prescribe medications. Do NOT provide dosages. \"",
                "                        \"Do NOT replace human medical judgment. \"",
                "                        \"Return ONLY valid JSON with the fields: question, normalized_answer.\"",
                "                    )",
                "                },",
                "                {",
                "                    \"role\": \"user\",",
                "                    \"content\": f\"Question: {question}\\nAnswer: {answer}\"",
                "                }",
                "            ],",
                "            temperature=0.3,",
                "            max_tokens=256,",
                "            top_p=1",
                "        )",
                "",
                "        # 3. Parse correto do JSON retornado",
                "        parsed_response = json.loads(response.choices[0].message.content)",
                "",
                "        normalized_results.append({",
                "            \"question\": parsed_response.get(\"question\", question),",
                "            \"normalized_answer\": parsed_response.get(\"normalized_answer\", parsed_response.get(\'normalized_answer\', \'\'))",
                "        })",
                "",
                "        print(f\"✔ Processed question: {question[:60]}...\")",
                "",
                "    # 4. Salva o resultado final",
                "    with open(output_file, \"w\", encoding=\"utf-8\") as out:",
                "        json.dump({\"medical_qa\": normalized_results}, out, ensure_ascii=False, indent=2)",
                "",
                "    print(f\"\\n✅ Dataset normalizado salvo em: {output_file}\")",
                "",
                "",
                "# Execução",
                "normalize_medical_qa(\"data/ori_pqal.json\")",
                ""
            ]
                "                {",
                "                    \"role\": \"user\",",
                "                    \"content\": f\"Question: {question}\\nAnswer: {answer}\"",
                "                }",
                "            ],",
                "            temperature=0.3,",
                "            max_tokens=256,",
                "            top_p=1",
                "        )",
                "",
                "        # 3. Parse correto do JSON retornado (SEM string replace)",
                "        parsed_response = json.loads(response.choices[0].message.content)",
                "",
                "        normalized_results.append({",
                "            \"question\": parsed_response[\"question\"],",
                "            \"normalized_answer\": parsed_response[\"normalized_answer\"]",
                "        })",
                "",
                "        # Opcional: log simples",
                "        print(f\"✔ Processed question: {question[:60]}...\")",
                "",
                "    # 4. Salva o resultado final",
                "    with open(output_file, \"w\", encoding=\"utf-8\") as out:",
                "        json.dump(",
                "            {\"medical_qa\": normalized_results},",
                "            out,",
                "            ensure_ascii=False,",
                "            indent=2",
                "        )",
                "",
                "    print(f\"\\n✅ Dataset normalizado salvo em: {output_file}\")",
                "",
                "",
                "# Execução",
                "normalize_medical_qa(\"data/ori_pqal.json\")",
                ""
            ]
        }
    ]
}

In [ ]:
#Conexão com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import requests
import json
from openai import OpenAI
from openai_key import key
# Substitua 'your_openai_api_key' pela sua chave de API da OpenAI
client = OpenAI(api_key=key)


In [4]:
import json

def normalize_medical_qa(input_file, output_file="normalized_medical_qa.json"):
    # 1. Carrega o dataset médico
    with open(input_file, "r", encoding="utf-8") as file:
        data = json.load(file)

    # AJUSTE AQUI conforme a estrutura real do seu JSON
    # Exemplo: data["data"], data["qas"], etc.
    qas = data["data"]

    normalized_results = []

    # 2. Processa cada pergunta e resposta
    for item in qas:
        question = item["question"]
        answer = item["answer"]

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            response_format={"type": "json_object"},
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a clinical decision-support assistant. "
                        "Rewrite the medical answer in a concise, objective, and clinically safe way. "
                        "Do NOT prescribe medications. Do NOT provide dosages. "
                        "Do NOT replace human medical judgment. "
                        "Return ONLY valid JSON with the fields: question, normalized_answer."
                    )
                },
                {
                    "role": "user",
                    "content": f"Question: {question}\nAnswer: {answer}"
                }
            ],
            temperature=0.3,
            max_tokens=256,
            top_p=1
        )

        # 3. Parse correto do JSON retornado (SEM string replace)
        parsed_response = json.loads(response.choices[0].message.content)

        normalized_results.append({
            "question": parsed_response["question"],
            "normalized_answer": parsed_response["normalized_answer"]
        })

        # Opcional: log simples
        print(f"✔ Processed question: {question[:60]}...")

    # 4. Salva o resultado final
    with open(output_file, "w", encoding="utf-8") as out:
        json.dump(
            {"medical_qa": normalized_results},
            out,
            ensure_ascii=False,
            indent=2
        )

    print(f"\n✅ Dataset normalizado salvo em: {output_file}")


# Execução
normalize_medical_qa("data/ori_pqal.json")


KeyError: 'data'